## Binding of `barrier` Regions

The binding rules call for a `barrier` region to bind to the closest enclosing  `parallel` region.

In the following example, the call from the main program to  _sub2_  is conforming  because the `barrier` region (in  _sub3_ ) binds to the `parallel`  region in  _sub2_ . The call from the main program to  _sub1_  is conforming  because the `barrier` region binds to the `parallel` region in subroutine   _sub2_ .

The call from the main program to  _sub3_  is conforming because the `barrier`  region binds to the implicit inactive `parallel` region enclosing the sequential  part. Also note that the `barrier` region in  _sub3_  when called from   _sub2_  only synchronizes the team of threads in the enclosing `parallel`  region and not all the threads created in  _sub1_ .

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: barrier_regions.1c
* type: C
*/
void work(int n) {}

void sub3(int n)
{
  work(n);
  #pragma omp barrier
  work(n);
}

void sub2(int k)
{
  #pragma omp parallel shared(k)
    sub3(k);
}

void sub1(int n)
{
  int i;
  #pragma omp parallel private(i) shared(n)
  {
    #pragma omp for
    for (i=0; i<n; i++)
      sub2(i);
  }
}

int main()
{
  sub1(2);
  sub2(2);
  sub3(2);
  return 0;
}



In [ ]:

! name: barrier_regions.1f
! type: F-fixed
      SUBROUTINE WORK(N)
        INTEGER N
      END SUBROUTINE WORK

      SUBROUTINE SUB3(N)
      INTEGER N
        CALL WORK(N)
!$OMP   BARRIER
        CALL WORK(N)
      END SUBROUTINE SUB3

      SUBROUTINE SUB2(K)
      INTEGER K
!$OMP   PARALLEL SHARED(K)
          CALL SUB3(K)
!$OMP   END PARALLEL
      END SUBROUTINE SUB2


      SUBROUTINE SUB1(N)
      INTEGER N
        INTEGER I
!$OMP   PARALLEL PRIVATE(I) SHARED(N)
!$OMP     DO
          DO I = 1, N
            CALL SUB2(I)
          END DO
!$OMP   END PARALLEL
      END SUBROUTINE SUB1

      PROGRAM EXAMPLE
        CALL SUB1(2)
        CALL SUB2(2)
        CALL SUB3(2)
      END PROGRAM EXAMPLE

